<a href="https://colab.research.google.com/github/LeonSilva15/fcc-data-analysis/blob/main/demographic_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import unittest

In [65]:
!wget 'https://raw.githubusercontent.com/LeonSilva15/fcc-data-analysis/main/data/adult.data.csv'

--2023-10-14 21:39:50--  https://raw.githubusercontent.com/LeonSilva15/fcc-data-analysis/main/data/adult.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3518607 (3.4M) [text/plain]
Saving to: ‘adult.data.csv.1’

adult.data.csv.1    100%[===================>]   3.36M  --.-KB/s    in 0.02s   

2023-10-14 21:39:50 (220 MB/s) - ‘adult.data.csv.1’ saved [3518607/3518607]



In [4]:
# Read data from file
df = pd.read_csv( 'adult.data.csv' )
print( df.info() )
print( df.head() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  salary          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov  

In [63]:
def calculate_demographic_data( print_data=True ):
    # How many of each race are represented in this dataset? This should be a Pandas series with race names as the index labels.
    race_count = df['race'].value_counts()

    # What is the average age of men?
    average_age_men = df.loc[ df['sex'] == 'Male', 'age' ].mean()

    # What is the percentage of people who have a Bachelor's degree?
    percentage_bachelors = df.loc[ df['education'] == 'Bachelors' ].shape[0] / df.shape[0] * 100

    # What percentage of people with advanced education (`Bachelors`, `Masters`, or `Doctorate`) make more than 50K?
    # What percentage of people without advanced education make more than 50K?

    # with and without `Bachelors`, `Masters`, or `Doctorate`
    higher_levels = ['Bachelors', 'Masters', 'Doctorate']

    # percentage with salary >50K
    higher_education_df = df[ df['education'].isin(  higher_levels ) ]
    higher_education_rich = higher_education_df[higher_education_df['salary'] == '>50K'].shape[0] / higher_education_df.shape[0] * 100

    lower_education_df = df[ ~df['education'].isin( higher_levels ) ]
    lower_education_rich = lower_education_df[ df['salary'] == '>50K' ].shape[0] / lower_education_df.shape[0] * 100

    # What is the minimum number of hours a person works per week (hours-per-week feature)?

    min_work_hours = df['hours-per-week'].min()

    # What percentage of the people who work the minimum number of hours per week have a salary of >50K?
    num_min_workers = df.loc[ (df['hours-per-week'] == df['hours-per-week'].min())].shape[0]
    rich_percentage = df.loc[ (df['hours-per-week'] == df['hours-per-week'].min()) & (df['salary'] == '>50K') ].shape[0] / num_min_workers * 100


    # What country has the highest percentage of people that earn >50K?
    country_earning_df = df[ df['salary'] == '>50K'][ 'native-country' ].value_counts()
    country_earning_percentage_df = country_earning_df / df[ 'native-country' ].value_counts()

    highest_earning_country = country_earning_percentage_df.idxmax()
    highest_earning_country_percentage = country_earning_percentage_df.max() * 100

    # Identify the most popular occupation for those who earn >50K in India.
    top_IN_occupation_df = df[ (df['native-country'] == 'India') & (df['salary'] == '>50K') ][ 'occupation' ].value_counts()
    top_IN_occupation = top_IN_occupation_df.idxmax()

    # DO NOT MODIFY BELOW THIS LINE

    if print_data:
        print("Number of each race:\n", race_count)
        print("Average age of men:", average_age_men)
        print(f"Percentage with Bachelors degrees: {percentage_bachelors}%")
        print(f"Percentage with higher education that earn >50K: {higher_education_rich}%")
        print(f"Percentage without higher education that earn >50K: {lower_education_rich}%")
        print(f"Min work time: {min_work_hours} hours/week")
        print(f"Percentage of rich among those who work fewest hours: {rich_percentage}%")
        print("Country with highest percentage of rich:", highest_earning_country)
        print(f"Highest percentage of rich people in country: {highest_earning_country_percentage}%")
        print("Top occupations in India:", top_IN_occupation)

    return {
        'race_count': race_count,
        'average_age_men': round(average_age_men,1),
        'percentage_bachelors': round(percentage_bachelors,1),
        'higher_education_rich': round(higher_education_rich,1),
        'lower_education_rich': round(lower_education_rich,1),
        'min_work_hours': min_work_hours,
        'rich_percentage': rich_percentage,
        'highest_earning_country': highest_earning_country,
        'highest_earning_country_percentage':
        round(highest_earning_country_percentage,1),
        'top_IN_occupation': top_IN_occupation
    }

# calculate_demographic_data()

In [54]:
class DemographicAnalyzerTestCase(unittest.TestCase):
    @classmethod
    def setUpClass(self):
        self.data = calculate_demographic_data(print_data = False)

    def test_race_count(self):
        actual = self.data['race_count'].tolist()
        expected = [27816, 3124, 1039, 311, 271]
        self.assertCountEqual(actual, expected, msg="Expected race count values to be [27816, 3124, 1039, 311, 271]")

    def test_average_age_men(self):
        actual = self.data['average_age_men']
        expected = 39.4
        self.assertAlmostEqual(actual, expected, msg="Expected different value for average age of men.")

    def test_percentage_bachelors(self):
        actual = self.data['percentage_bachelors']
        expected = 16.4
        self.assertAlmostEqual(actual, expected, msg="Expected different value for percentage with Bachelors degrees.")

    def test_higher_education_rich(self):
        actual = self.data['higher_education_rich']
        expected = 46.5
        self.assertAlmostEqual(actual, expected, msg="Expected different value for percentage with higher education that earn >50K.")

    def test_lower_education_rich(self):
        actual = self.data['lower_education_rich']
        expected = 17.4
        self.assertAlmostEqual(actual, expected, msg="Expected different value for percentage without higher education that earn >50K.")

    def test_min_work_hours(self):
        actual = self.data['min_work_hours']
        expected = 1
        self.assertAlmostEqual(actual, expected, msg="Expected different value for minimum work hours.")

    def test_rich_percentage(self):
        actual = self.data['rich_percentage']
        expected = 10
        self.assertAlmostEqual(actual, expected, msg="Expected different value for percentage of rich among those who work fewest hours.")

    def test_highest_earning_country(self):
        actual = self.data['highest_earning_country']
        expected = 'Iran'
        self.assertEqual(actual, expected, "Expected different value for highest earning country.")

    def test_highest_earning_country_percentage(self):
        actual = self.data['highest_earning_country_percentage']
        expected = 41.9
        self.assertAlmostEqual(actual, expected, msg="Expected different value for highest earning country percentage.")

    def test_top_IN_occupation(self):
        actual = self.data['top_IN_occupation']
        expected = 'Prof-specialty'
        self.assertEqual(actual, expected, "Expected different value for top occupations in India.")

In [64]:
# Run unit tests
unittest.main( argv=[''], verbosity=2, exit=False )

<ipython-input-63-9e407d5e0c36>:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lower_education_rich = lower_education_df[ df['salary'] == '>50K' ].shape[0] / lower_education_df.shape[0] * 100
test_average_age_men (__main__.DemographicAnalyzerTestCase) ... ok
test_higher_education_rich (__main__.DemographicAnalyzerTestCase) ... ok
test_highest_earning_country (__main__.DemographicAnalyzerTestCase) ... ok
test_highest_earning_country_percentage (__main__.DemographicAnalyzerTestCase) ... ok
test_lower_education_rich (__main__.DemographicAnalyzerTestCase) ... ok
test_min_work_hours (__main__.DemographicAnalyzerTestCase) ... ok
test_percentage_bachelors (__main__.DemographicAnalyzerTestCase) ... ok
test_race_count (__main__.DemographicAnalyzerTestCase) ... ok
test_rich_percentage (__main__.DemographicAnalyzerTestCase) ... ok
test_top_IN_occupation (__main__.DemographicAnalyzerTestCase) ... ok

---------------------------------------------------------------